In [319]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0
import folium

### Information below contains a chart with consolidated Neighborhood based on Borough and Zip code. Data was obtained from wikipedia. At the following link https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.


In [320]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') # download data
df = df[0]# Select appropriate index for data
df = df[df['Borough']!='Not assigned'] # Remove all boroughs that don't have an assigned zip code
df['Borough'].unique() # checking no other data anomilies exist
df.groupby(['Postcode','Borough'], as_index = False, sort = False).agg(','.join) # Comebine neighborhoods within the same zip code and Borough
df['Neighbourhood'] = np.where(df["Neighbourhood"] == 'Not assigned', df['Borough'], df['Neighbourhood']) # Missing values in Neighborhood replaced with Borough.
df.shape

(211, 3)

### Below we obtain a separate file which provides longitude and latitude of specific zip codes, we will merge the existing table above with the new information


In [321]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [322]:
df2 = pd.read_csv('http://cocl.us/Geospatial_data')
df = pd.merge(df,df2,how = 'left', left_on=df['Postcode'], right_on=df2['Postal Code'])
df = df[['Postcode','Borough', 'Neighbourhood', 'Latitude','Longitude']]
df.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


### Create a map of Toronto

In [323]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Toronto_Location")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [324]:
# create map of Toronto using latitude and longitude values
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
    
Toronto_map

### There seems to be a high density of points located at Downtown Toronto, I will be concentrating on that are for further exploration.

In [325]:
df_toronto = df[df['Borough'] == 'Downtown Toronto']

In [326]:
df_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
13,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
14,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
27,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [327]:
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronro are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronro are 43.6541737, -79.3808116451341.


In [328]:
# create map of Toronto using latitude and longitude values
Dtown_Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Dtown_Toronto_map)  
    
Dtown_Toronto_map

### Define Foursquare Credentials

In [329]:
CLIENT_ID = 'BWU4F1HVP4WGYXL235HO1YKGGM1ZH1CC130AVLPHR0DRHH52' # your Foursquare ID
CLIENT_SECRET = 'ZDLJGFVIQQ2OPDD5DN1VKPM30PDBPY0S0HLJRFGEVMVQZQ54' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


### Obtain venues top 20 venues within 100 meters


In [330]:
LIMIT = 20 #limit for venues defined
def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [331]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Harbourfront East
Toronto Islands
Union Station
Design Exchange
Toronto Dominion Centre
Commerce Court
Victoria Hotel
Harbord
University of Toronto
Chinatown
Grange Park
Kensington Market
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


### Resulting data frame

In [332]:
toronto_venues.head()

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
1,Harbourfront,43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
2,Regent Park,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
3,Regent Park,43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Ryerson,43.657162,-79.378937,Ryerson Image Centre,43.657523,-79.379460,Art Gallery


### Grouping of each category within a neighborhood and standardizing the frequency of each occurancy by average

In [333]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head(3)

,Neighbourhood,American Restaurant,Art Gallery,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Building,Burger Joint,...,Sandwich Place,Seafood Restaurant,Soup Place,Spa,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Thrift / Vintage Store,Vegetarian / Vegan Restaurant
0,Adelaide,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.083333,0.083333,0.083333,0.0,0.0,0.083333
1,Bathurst Quay,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
2,CN Tower,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000


In [334]:
toronto_onehot.head()

,Neighbourhood,American Restaurant,Art Gallery,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Building,Burger Joint,...,Sandwich Place,Seafood Restaurant,Soup Place,Spa,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Thrift / Vintage Store,Vegetarian / Vegan Restaurant
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Regent Park,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,Regent Park,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ryerson,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Most common venue 20 venues

In [335]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Adelaide,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bar,Coffee Shop,Concert Hall,Food Court,Garden,Hotel,Greek Restaurant,...,Sushi Restaurant,Tea Room,Burrito Place,Deli / Bodega,Cupcake Shop,College Gym,Cocktail Bar,Café,Sandwich Place,Farmers Market
1,Bathurst Quay,Performing Arts Venue,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
2,CN Tower,Performing Arts Venue,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
3,Cabbagetown,Italian Restaurant,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
4,Central Bay Street,Coffee Shop,Sandwich Place,Vegetarian / Vegan Restaurant,Cocktail Bar,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,Concert Hall,College Gym,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar


### Cluster Neighborhoods N=7

In [336]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged = toronto_merged.dropna() # drop any neighborhoods that didnt have any data

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,3.0,Spa,Breakfast Spot,Vegetarian / Vegan Restaurant,Cocktail Bar,...,Concert Hall,College Gym,Coffee Shop,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Bookstore,Bar
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,3.0,Spa,Breakfast Spot,Vegetarian / Vegan Restaurant,Cocktail Bar,...,Concert Hall,College Gym,Coffee Shop,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Bookstore,Bar
13,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,1.0,Art Gallery,Coffee Shop,Vegetarian / Vegan Restaurant,Cocktail Bar,...,Concert Hall,College Gym,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
14,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,1.0,Art Gallery,Coffee Shop,Vegetarian / Vegan Restaurant,Cocktail Bar,...,Concert Hall,College Gym,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
27,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3.0,Italian Restaurant,Japanese Restaurant,Performing Arts Venue,Hostel,...,Concert Hall,College Gym,Cocktail Bar,Garden,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore


### Create a cluster map

In [337]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster examination

In [338]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
#Looks like a very healthy neighborhood

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
61,Harbourfront East,0.0,Gym,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
62,Toronto Islands,0.0,Gym,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
63,Union Station,0.0,Gym,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar


In [339]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
#Nothing too special about this neighborhood

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
13,Ryerson,1.0,Art Gallery,Coffee Shop,Vegetarian / Vegan Restaurant,Cocktail Bar,Garden,Food Court,Fast Food Restaurant,Farmers Market,...,Concert Hall,College Gym,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
14,Garden District,1.0,Art Gallery,Coffee Shop,Vegetarian / Vegan Restaurant,Cocktail Bar,Garden,Food Court,Fast Food Restaurant,Farmers Market,...,Concert Hall,College Gym,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
41,Central Bay Street,1.0,Coffee Shop,Sandwich Place,Vegetarian / Vegan Restaurant,Cocktail Bar,Garden,Food Court,Fast Food Restaurant,Farmers Market,...,Concert Hall,College Gym,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
74,Design Exchange,1.0,Coffee Shop,Restaurant,Gym,Deli / Bodega,Vegetarian / Vegan Restaurant,Cocktail Bar,Food Court,Fast Food Restaurant,...,Concert Hall,College Gym,Café,Gastropub,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
75,Toronto Dominion Centre,1.0,Coffee Shop,Restaurant,Gym,Deli / Bodega,Vegetarian / Vegan Restaurant,Cocktail Bar,Food Court,Fast Food Restaurant,...,Concert Hall,College Gym,Café,Gastropub,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar


In [340]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
#Art/Performing Hall centered neighborhood

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
164,CN Tower,2.0,Performing Arts Venue,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
165,Bathurst Quay,2.0,Performing Arts Venue,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
166,Island airport,2.0,Performing Arts Venue,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
167,Harbourfront West,2.0,Performing Arts Venue,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
168,King and Spadina,2.0,Performing Arts Venue,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
169,Railway Lands,2.0,Performing Arts Venue,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
170,South Niagara,2.0,Performing Arts Venue,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar


In [341]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
#THis neighborhood probably has the most active night life

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
2,Harbourfront,3.0,Spa,Breakfast Spot,Vegetarian / Vegan Restaurant,Cocktail Bar,Garden,Food Court,Fast Food Restaurant,Farmers Market,...,Concert Hall,College Gym,Coffee Shop,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Bookstore,Bar
3,Regent Park,3.0,Spa,Breakfast Spot,Vegetarian / Vegan Restaurant,Cocktail Bar,Garden,Food Court,Fast Food Restaurant,Farmers Market,...,Concert Hall,College Gym,Coffee Shop,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Bookstore,Bar
27,St. James Town,3.0,Italian Restaurant,Japanese Restaurant,Performing Arts Venue,Hostel,Coffee Shop,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Farmers Market,...,Concert Hall,College Gym,Cocktail Bar,Garden,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore
42,Christie,3.0,Nightclub,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
49,Adelaide,3.0,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bar,Coffee Shop,Concert Hall,Food Court,Garden,Hotel,...,Sushi Restaurant,Tea Room,Burrito Place,Deli / Bodega,Cupcake Shop,College Gym,Cocktail Bar,Café,Sandwich Place,Farmers Market
50,King,3.0,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bar,Coffee Shop,Concert Hall,Food Court,Garden,Hotel,...,Sushi Restaurant,Tea Room,Burrito Place,Deli / Bodega,Cupcake Shop,College Gym,Cocktail Bar,Café,Sandwich Place,Farmers Market
51,Richmond,3.0,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bar,Coffee Shop,Concert Hall,Food Court,Garden,Hotel,...,Sushi Restaurant,Tea Room,Burrito Place,Deli / Bodega,Cupcake Shop,College Gym,Cocktail Bar,Café,Sandwich Place,Farmers Market
87,Commerce Court,3.0,American Restaurant,Salad Place,Deli / Bodega,Gym,Coffee Shop,Café,Burrito Place,Pub,...,Sandwich Place,Soup Place,Art Gallery,Bakery,Seafood Restaurant,Sushi Restaurant,College Gym,Farmers Market,Cupcake Shop,Concert Hall
88,Victoria Hotel,3.0,American Restaurant,Salad Place,Deli / Bodega,Gym,Coffee Shop,Café,Burrito Place,Pub,...,Sandwich Place,Soup Place,Art Gallery,Bakery,Seafood Restaurant,Sushi Restaurant,College Gym,Farmers Market,Cupcake Shop,Concert Hall
152,Chinatown,3.0,Cocktail Bar,Café,Farmers Market,Liquor Store,Thrift / Vintage Store,Bar,Concert Hall,Garden,...,Art Gallery,Deli / Bodega,Cupcake Shop,Bakery,College Gym,Bookstore,Coffee Shop,Gastropub,Bank,Burrito Place


In [342]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
#Food concentrated area

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
189,Cabbagetown,4.0,Italian Restaurant,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar


In [343]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
#College town

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
143,Harbord,5.0,College Gym,Vegetarian / Vegan Restaurant,Cocktail Bar,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,Coffee Shop,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar,Bank
144,University of Toronto,5.0,College Gym,Vegetarian / Vegan Restaurant,Cocktail Bar,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,Coffee Shop,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar,Bank


In [344]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
#Recreational area

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
183,Rosedale,6.0,Park,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
